# Twitter Data Wrangling

## Table of Contents
<ul>
<li><a href="#Intro">Introduction</a></li>
<li><a href="#Gather">Gather</a></li>
<li><a href="#Assess">Assess</a></li>
<li><a href="#Clean">Clean</a></li>
<li><a href="#Store">Store</a></li>
</ul>

<a id='Intro'></a>
## Introduction

> Real-world data hardly comes clean. This project mainly focuses on gathering data from a variety of sources and in a variety of formats, assess its quality and tidiness, then clean it. The dataset that I wrangle is the tweet archive of Twitter user @dog_rates, (a.k.a WeRateDogs). “WeRateDogs” is a Twitter account that rates people's dogs with a humorous comment about the dog. The account was started in 2015 by college student Matt Nelson, has nearly 7.6 million followers, as of December 2018. Specifically, three datasets are gathered as described below.
1.	The WeRateDogs Twitter archive. The file is provided from “WeRateDogs” for this project. It has a csv format.
2.	The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file is programmatically downloaded from Udacity's servers using URL. 
3.	Additionally, each tweet's retweet count and favourite ("like") count data is gathered from the Twitter API. 

<a id='Gather'></a>
## Gather

> Gathering data from different sources.

### Source 1: Flat file

In [1]:
import pandas as pd 
archive_df = pd.read_csv('twitter-archive-enhanced.csv')

### Source 2:  URL

In [2]:
import requests
import os 
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
response 

<Response [200]>

In [3]:
folder_name = r'C:\Users\wsy\Doc\Udacity\Data Analytics\Data Wrangling\Project'

with open(os.path.join(folder_name, url.split('/')[-1]), mode = 'wb') as file: 
    file.write(response.content)

In [4]:
predictions_df = pd.read_csv('image-predictions.tsv', sep='\t')

### Source 3: twitter API (tweepy)

In [5]:
# Import Python's Tweepy library
import tweepy
print(tweepy.__version__)

3.9.0


In [6]:
consumer_key = 'PERSONAL_CONSUMER_KEY'
consumer_secret = 'PERSONAL_CONSUMER_SECRET'

access_token = 'PERSONAL_ACCESS_TOKEN'
access_secret = 'PERSONAL_ACCESS_SECRET'

# Connect to twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [7]:
tweet_ids = archive_df.tweet_id
len(tweet_ids)

2356

In [8]:
import json
import time

count = 0
fails_dict = {}
t = time.process_time()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
elapsed_time = time.process_time() - t

1: 892420643555336193
Success
2: 892177421306343426
Success
3: 891815181378084864
Success
4: 891689557279858688
Success
5: 891327558926688256
Success
6: 891087950875897856
Success
7: 890971913173991426
Success
8: 890729181411237888
Success
9: 890609185150312448
Success
10: 890240255349198849
Success
...2351: 666050758794694657
Success
2352: 666049248165822465
Success
2353: 666044226329800704
Success
2354: 666033412701032449
Success
2355: 666029285002620928
Success
2356: 666020888022790149
Success


In [9]:
print(elapsed_time)
print(fails_dict)

46.328125
{888202515573088257: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 873697596434513921: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 872668790621863937: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 872261713294495745: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 869988702071779329: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 866816280283807744: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 861769973181624320: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 856602993587888130: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 851953902622658560: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 845459076796616705: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 844704788403113984: TweepError([{'code': 144, 'message': 'N

In [10]:
json_df=pd.DataFrame(columns=['tweet_id','retweet_count','favorite_count'])
with open('tweet_json.txt') as json_file:
    for line in json_file:
        status  = json.loads(line)
        tweet_id = status['id_str']
        retweet_count = status['retweet_count']
        favorite_count = status['favorite_count']
                
        json_df = json_df.append(pd.DataFrame([[tweet_id, retweet_count, favorite_count]],
                                              columns=['tweet_id','retweet_count','favorite_count']))
json_df = json_df.reset_index(drop=True)

<a id='Assess'></a>
## Assess
>   <li><a href="#Tidiness"> 1. Quality Assessment</a></li>
   <li><a href="#Tidiness"> 2. Tidiness Assessment</a></li>

### 1. Quality 
#### 1.1. 'archive_df' **Table**

In [11]:
archive_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [12]:
archive_df.shape # 2356 data points and 17 columns

(2356, 17)

In [13]:
archive_df.info() # Check the datatype of each column in the table (archive_df) & missing values 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [14]:
sum(archive_df.duplicated(subset=['tweet_id'])) # Check the duplicates 

0

In [15]:
archive_df.source.value_counts() # This column should be cleaned up!

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [16]:
archive_df.name.value_counts()

None         745
a             55
Charlie       12
Oliver        11
Cooper        11
            ... 
Enchilada      1
Superpup       1
Leonidas       1
Stuart         1
Grizz          1
Name: name, Length: 957, dtype: int64

> **Quality assessments of  _archive_df_**
- 'Tweet_id' is an integer not a string.
- 'Timestamp' column should be "datetime" datatype. 
- 'Source' column can be simplified. 
- Some columns such as 'doggo', 'floofer', 'pupper', 'puppo' and 'name' have missing values, but denoted as _None_, instead of _NaN_. 
- Some names of dogs have its name 'a', it should be changed to _NaN_. 
- The columns containing lots of missing values ('in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp') should be dropped if it is not necessary for the analysis. Otherwise, it should be imputed. 

#### 1.2. 'predictions_df' **Table**

In [17]:
predictions_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [18]:
predictions_df.info() # Check the datatype of each column in the table (predictions_df) & missing values 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [19]:
predictions_df.p1.value_counts()

golden_retriever      150
Labrador_retriever    100
Pembroke               89
Chihuahua              83
pug                    57
                     ... 
bookcase                1
radio_telescope         1
picket_fence            1
sandbar                 1
zebra                   1
Name: p1, Length: 378, dtype: int64

In [20]:
sum(predictions_df.duplicated()) # Check the duplicates 

0

> **Quality assessments of 'predictions_df' Table**
- Tweet_id is an integer not a string.
- The letters in the columns: p1, p2, p3 are inconsistent.
- The underscore between the two words should be removed.

#### 1.3. 'json_df' **Table**

In [21]:
json_df.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7561,35745
1,892177421306343426,5601,30882
2,891815181378084864,3712,23230
3,891689557279858688,7745,39049
4,891327558926688256,8340,37292


In [22]:
json_df.info() # Check the datatype of each column in the table (json_df) & missing values 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2331 non-null   object
 1   retweet_count   2331 non-null   object
 2   favorite_count  2331 non-null   object
dtypes: object(3)
memory usage: 54.8+ KB


> **Quality assessments of 'json_df' Table**
- The datatype of retweet_count & favorite_count is string. It should be integer.

<a id='Tidiness'></a>
### 2. Tidiness

- The column 'expanded_urls' from archive_df has duplicated infomation with 'tweet_id'.
- Doggo, floofer, pupper, and puppo columns in archive_df table should be replaced with a single variable. Then the single variable should be merged with the table archive_df.
- 'json_df' has similar infomation with the column 'ratings' in 'archive_df'. Both dataframes should be combined.

<a id='Clean'></a>
## Clean

> Before the cleaning occurs, I made a copy of each piece of data. All of the cleaning operations will be conducted on this copy so I can still view the original dirty and/or messy dataset later.

In [23]:
archive_df_clean = archive_df.copy()
predictions_df_clean = predictions_df.copy()
json_df_clean = json_df.copy()

### 1. archive_df

#### 1.1. Define 
- Change the datatype of 'tweet_id' to string. 
- Change the datatype of 'timestamp' column as datetime.

#### Code

In [24]:
archive_df_clean.tweet_id = archive_df_clean.tweet_id.astype(str)
archive_df_clean['timestamp'] = pd.to_datetime(archive_df_clean['timestamp'])

#### Test

In [25]:
print (archive_df_clean.tweet_id.dtype)
print (archive_df_clean.timestamp.dtype)

object
datetime64[ns, UTC]


#### 1.2. Define
- Clean up 'Source' column more concisely.

#### Code

In [26]:
archive_df_clean['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [27]:
archive_df_clean['source'] = archive_df_clean['source'].str.replace('<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'Twitter for iPhone')
archive_df_clean['source'] = archive_df_clean['source'].str.replace('<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>','Vine - Make a Scene')
archive_df_clean['source'] = archive_df_clean['source'].str.replace('<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>','Twitter Web Client')
archive_df_clean['source'] = archive_df_clean['source'].str.replace('<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>','TweetDeck')

#### Test

In [28]:
archive_df_clean['source'].value_counts()

Twitter for iPhone     2221
Vine - Make a Scene      91
Twitter Web Client       33
TweetDeck                11
Name: source, dtype: int64

#### 1.3. Define
- Replace any "None" object values in dataframe to 'NaN'. 
- Replace the data points of 'a' in the 'name' column with 'NaN'.

#### Code

In [29]:
import numpy as np

In [30]:
archive_df_clean.replace('None', np.NaN, inplace = True)
archive_df_clean.name.replace('a', np.NaN, inplace = True)

#### Test

In [31]:
archive_df_clean.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56+00:00,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,NaN,NaN,NaN,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27+00:00,Twitter for iPhone,This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,NaN,NaN,NaN,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03+00:00,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,NaN,NaN,NaN,NaN


In [32]:
archive_df_clean.query('name == "a"') # All the data points with the name : "a" is changed to "NaN". 

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


### 2. predictions_df

#### 2.1. Define 
- Change the datatype of 'Tweet_id' to string. 

#### Code

In [33]:
predictions_df_clean['tweet_id']=predictions_df_clean['tweet_id'].astype('object')

#### Test 

In [34]:
predictions_df_clean.tweet_id.dtype

dtype('O')

#### 2.2. Define 
- Remove the underscore between two words in the columns: p1, p2, p3. 
- Convert the names to uppercase for the first letter and lowercase for the rest in the columns: p1, p2, p3.

#### Code

In [35]:
predictions_df_clean['p1'] = predictions_df_clean['p1'].str.replace('_', ' ')
predictions_df_clean['p2'] = predictions_df_clean['p2'].str.replace('_', ' ')
predictions_df_clean['p3'] = predictions_df_clean['p3'].str.replace('_', ' ')

In [36]:
predictions_df_clean['p1'] = predictions_df_clean['p1'].str.title()
predictions_df_clean['p2'] = predictions_df_clean['p2'].str.title()
predictions_df_clean['p3'] = predictions_df_clean['p3'].str.title()

#### Test

In [37]:
predictions_df_clean.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh Springer Spaniel,0.465074,True,Collie,0.156665,True,Shetland Sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,Redbone,0.506826,True,Miniature Pinscher,0.074192,True,Rhodesian Ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German Shepherd,0.596461,True,Malinois,0.138584,True,Bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian Ridgeback,0.408143,True,Redbone,0.360687,True,Miniature Pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,Miniature Pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


### 3. json_df

#### 3.1. Define
- Change the datatype of retweet_count and favorite_count into integer. 

#### Code

In [38]:
json_df_clean['retweet_count'] = pd.to_numeric(json_df_clean['retweet_count'])
json_df_clean['favorite_count'] = pd.to_numeric(json_df_clean['favorite_count'])

#### Test 

In [39]:
json_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2331 non-null   object
 1   retweet_count   2331 non-null   int64 
 2   favorite_count  2331 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 54.8+ KB


### 4. Tidiness

#### 4.1. Defined issues
- Drop the 'expanded_urls' from archive_df can be also dropped, since it has duplicated info with the column 'tweet_id'.
- Drop the unnecessary columns for the analysis: 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'.

#### Code

In [40]:
archive_df_clean.drop(['expanded_urls','in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], inplace = True, axis = 1)

In [41]:
archive_df_clean.head()

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,2017-08-01 16:23:56+00:00,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,NaN,NaN,NaN,NaN
1,892177421306343426,2017-08-01 00:17:27+00:00,Twitter for iPhone,This is Tilly. She's just checking pup on you....,13,10,Tilly,NaN,NaN,NaN,NaN
2,891815181378084864,2017-07-31 00:18:03+00:00,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,NaN,NaN,NaN,NaN
3,891689557279858688,2017-07-30 15:58:51+00:00,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,13,10,Darla,NaN,NaN,NaN,NaN
4,891327558926688256,2017-07-29 16:00:24+00:00,Twitter for iPhone,This is Franklin. He would like you to stop ca...,12,10,Franklin,NaN,NaN,NaN,NaN


#### Test

In [42]:
archive_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            2356 non-null   object             
 1   timestamp           2356 non-null   datetime64[ns, UTC]
 2   source              2356 non-null   object             
 3   text                2356 non-null   object             
 4   rating_numerator    2356 non-null   int64              
 5   rating_denominator  2356 non-null   int64              
 6   name                1556 non-null   object             
 7   doggo               97 non-null     object             
 8   floofer             10 non-null     object             
 9   pupper              257 non-null    object             
 10  puppo               30 non-null     object             
dtypes: datetime64[ns, UTC](1), int64(2), object(8)
memory usage: 202.6+ KB


#### 4.2. Defined issues
- Replace Doggo, floofer, pupper, and puppo columns in archive_df table with a single variable.
- Then merge the single variable with the table archive_df_clean.

#### Code

In [43]:
archive_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            2356 non-null   object             
 1   timestamp           2356 non-null   datetime64[ns, UTC]
 2   source              2356 non-null   object             
 3   text                2356 non-null   object             
 4   rating_numerator    2356 non-null   int64              
 5   rating_denominator  2356 non-null   int64              
 6   name                1556 non-null   object             
 7   doggo               97 non-null     object             
 8   floofer             10 non-null     object             
 9   pupper              257 non-null    object             
 10  puppo               30 non-null     object             
dtypes: datetime64[ns, UTC](1), int64(2), object(8)
memory usage: 202.6+ KB


In [44]:
archive_df_melt = pd.melt(archive_df_clean,
                            id_vars = ['tweet_id'],
                            var_name='variable',
                            value_vars = ['doggo', 'floofer', 'pupper', 'puppo'])

In [45]:
archive_df_melt.head(5) 

,tweet_id,variable,value
0,892420643555336193,doggo,NaN
1,892177421306343426,doggo,NaN
2,891815181378084864,doggo,NaN
3,891689557279858688,doggo,NaN
4,891327558926688256,doggo,NaN


In [46]:
archive_df_melt.drop(['variable'], inplace = True, axis = 1)

In [47]:
archive_df_melt.drop_duplicates(inplace = True)

In [48]:
archive_df_melt.info() 
# Since there are 2356 unique tweet_ids in archive_df_clean, we can imply that there are 394 duplicates tweet_ids in archive_df_melt. 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2750 entries, 0 to 8151
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  2750 non-null   object
 1   value     394 non-null    object
dtypes: object(2)
memory usage: 64.5+ KB


In [49]:
# Drop the duplicated tweet_ids that its data point in value column is missing
archive_df_melt = archive_df_melt[~archive_df_melt[['tweet_id']].duplicated(keep=False) | archive_df_melt[['value']].notnull().any(axis=1)]

In [50]:
archive_df_melt.shape
# This indicates that there are some duplicated tweet_ids that have 2 values of dog type.

(2370, 2)

In [51]:
sum(archive_df_melt.duplicated(subset=['tweet_id'])) 
# 14 tweet_ids have 2 dog types.

14

In [52]:
archive_df_melt[archive_df_melt['tweet_id'].duplicated()]

,tweet_id,value
2556,854010172552949760,floofer
5172,817777686764523521,pupper
5243,808106460588765185,pupper
5277,802265048156610565,pupper
5287,801115127852503040,pupper
5417,785639753186217984,pupper
5445,781308096455073793,pupper
5490,775898661951791106,pupper
5534,770093767776997377,pupper
5601,759793422261743616,pupper


In [53]:
archive_df_melt.query('tweet_id =="854010172552949760"')

,tweet_id,value
200,854010172552949760,doggo
2556,854010172552949760,floofer


In [54]:
archive_df_clean.query('tweet_id =="854010172552949760"')
# The tweet_id 854010172552949760 is marked in 2 columns: doggo and floofer.

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
200,854010172552949760,2017-04-17 16:34:26+00:00,Twitter for iPhone,"At first I thought this was a shy doggo, but i...",11,10,NaN,doggo,floofer,NaN,NaN


In [55]:
archive_df_melt= archive_df_melt.drop_duplicates(subset=['tweet_id'], keep = 'last') 
# I decided to keep the one with last index. 

In [56]:
archive_df_melt.rename(columns={"value": "dog_stage"},inplace=True) # rename of the column 'value' as 'dog_stage'. 

In [57]:
archive_df_clean = archive_df_clean.merge(archive_df_melt, how = 'left')

In [58]:
columns = [ 'doggo', 'floofer', 'pupper', 'puppo']
archive_df_clean = archive_df_clean.drop(columns, axis = 1)

In [59]:
archive_df_clean.head()

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,dog_stage
0,892420643555336193,2017-08-01 16:23:56+00:00,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,NaN
1,892177421306343426,2017-08-01 00:17:27+00:00,Twitter for iPhone,This is Tilly. She's just checking pup on you....,13,10,Tilly,NaN
2,891815181378084864,2017-07-31 00:18:03+00:00,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,NaN
3,891689557279858688,2017-07-30 15:58:51+00:00,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,13,10,Darla,NaN
4,891327558926688256,2017-07-29 16:00:24+00:00,Twitter for iPhone,This is Franklin. He would like you to stop ca...,12,10,Franklin,NaN


#### 4.3 Defined issues 
- Merge 'json_df_clean' with 'archive_df_clean'.

#### Code

In [60]:
archive_df_clean.tweet_id = archive_df_clean.tweet_id.astype(str)
json_df_clean.tweet_id = json_df_clean.tweet_id.astype(str)

In [61]:
archive_df_clean_join = archive_df_clean.merge(json_df_clean, on = 'tweet_id', how = 'left')

In [62]:
archive_df_clean_join.shape

(2356, 10)

In [63]:
archive_df_clean_join.head()

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,dog_stage,retweet_count,favorite_count
0,892420643555336193,2017-08-01 16:23:56+00:00,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,13,10,Phineas,NaN,7561.0,35745.0
1,892177421306343426,2017-08-01 00:17:27+00:00,Twitter for iPhone,This is Tilly. She's just checking pup on you....,13,10,Tilly,NaN,5601.0,30882.0
2,891815181378084864,2017-07-31 00:18:03+00:00,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,12,10,Archie,NaN,3712.0,23230.0
3,891689557279858688,2017-07-30 15:58:51+00:00,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,13,10,Darla,NaN,7745.0,39049.0
4,891327558926688256,2017-07-29 16:00:24+00:00,Twitter for iPhone,This is Franklin. He would like you to stop ca...,12,10,Franklin,NaN,8340.0,37292.0


In [64]:
archive_df_clean_join.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            2356 non-null   object             
 1   timestamp           2356 non-null   datetime64[ns, UTC]
 2   source              2356 non-null   object             
 3   text                2356 non-null   object             
 4   rating_numerator    2356 non-null   int64              
 5   rating_denominator  2356 non-null   int64              
 6   name                1556 non-null   object             
 7   dog_stage           380 non-null    object             
 8   retweet_count       2331 non-null   float64            
 9   favorite_count      2331 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(2), int64(2), object(5)
memory usage: 202.5+ KB


<a id='Store'></a>
## Store

In [65]:
# Store the cleaned data with a csv format
archive_df_clean_join.to_csv('twitter_archive_master.csv',index=False)
predictions_df_clean.to_csv('predictions_master.csv',index=False)